In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
#https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook.
json_columns = ['device', 'geoNetwork','totals', 'trafficSource']  #these 4 columns are of type jspn objects
def load_dataframe(filename):
    df = pd.read_csv(filename, converters={column: json.loads for column in json_columns}, 
                     dtype={'fullVisitorId': 'str'}) #loading json columns and specifying the type of fullvisitor id as string
    
    for column in json_columns:
        column_as_df = json_normalize(df[column])  #normalizing json columns
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]  #creating subcolumns of json columns 
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)  #adding subcolumns and dropping the json columns
    return df

In [ ]:
test_data_conv = load_dataframe('test.csv')

In [ ]:
#https://docs.python.org/3/library/datetime.html
#adding datetime column in data
def add_date_features(df):
    df['date'] = df['date'].astype(str)
    df["date"] = df["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
    df["date"] = pd.to_datetime(df["date"])
    
    df["month"]   = df['date'].dt.month  #getting month
    df["day"]     = df['date'].dt.day   #getting day
    df["weekday"] = df['date'].dt.weekday   #getting date
    return df 

In [ ]:
test_data_conv = add_date_features(test_data_conv)

In [ ]:
## find constant columns
constant_columns = [column for column in test_data_conv.columns if test_data_conv[column].nunique(dropna=False)==1]

#dropping constant columns
test_data_conv_dropped = test_data_conv.drop(columns=constant_columns)

#Sorting by date to perform time based slicing
sorted_test_data_conv_dropped = test_data_conv_dropped.sort_values(by='date',ascending=True)

## non relevant columns
non_relevant = ["visitNumber", "date", "fullVisitorId", "sessionId", "visitId", "visitStartTime"]

## Drpiing non relevant columns
test = sorted_test_data_conv_dropped.drop(columns=non_relevant)

print("Before dropping the columns: ", test_data_conv.shape)
print("After dropping the columns: ", test.shape)

In [ ]:
categorical_columns = [column for column in test.columns if not column.startswith('total')]
categorical_columns = [column for column in categorical_features_train if column not in constant_columns + non_relevant]

for column in categorical_columns:

    le = LabelEncoder()
    
    test_values = list(test[column].values.astype(str))
    
    le.fit(test_values)

    test[column] = le.transform(test_values) 
    
test['device_isMobile'] = le.transform(test_values)

In [ ]:
#filling the NA values in totals column
def normalize_numerical_columns(dataframe, isTrainDataset = True):
    dataframe["totals_hits"] = dataframe["totals_hits"].astype(float)
    #dataframe["totals.hits"] = (dataframe["totals.hits"] - min(dataframe["totals.hits"])) / (max(dataframe["totals.hits"]) - min(dataframe["totals.hits"]))

    dataframe["totals_pageviews"] = dataframe["totals_pageviews"].astype(float)
    #dataframe["totals.pageviews"] = (dataframe["totals.pageviews"] - min(dataframe["totals.pageviews"])) / (max(dataframe["totals.pageviews"]) - min(dataframe["totals.pageviews"]))
    
    dataframe["totals_bounces"] = dataframe["totals_bounces"].astype(float)
    dataframe["totals_newVisits"] = dataframe["totals_newVisits"].astype(float)
    
    
    if isTrainDataset:
        dataframe["totals_transactionRevenue"] = dataframe["totals_transactionRevenue"].fillna(0.0)
    return dataframe 

test = normalize_numerical_columns(test,isTrainDataset=False)

In [ ]:
X_test = X_test.fillna(0).astype('float32')

In [ ]:
X_test['mean_pageViews_per_networkDomain'] = X_test.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('mean').astype('int')
X_test['mean_hits_per_networkDomain'] = X_test.groupby('geoNetwork_networkDomain')['totals_hits'].transform('mean').astype('int')